In [23]:
from vncorenlp import VnCoreNLP
from utils import *
import pandas as pd
import json

In [2]:
# Connect VncoreNLP server
annotator = VnCoreNLP(address="http://127.0.0.1", port=9000)

# Get pattern
pattern_path = '/Users/khanhnguyen/Document/QG/pattern/pattern.txt'
with open(pattern_path, "r") as f:
    patterns = f.read().split('\n')

In [14]:
def ques_gen(text):
    annotated_text = annotator.annotate(text)
    annotated_text = annotated_text['sentences']

    # Format annotated_text to old version
    tmp = {}
    for i, at in enumerate(annotated_text):
        tmp[i] = at
    annotated_text = tmp

    return question_generation(annotated_text, patterns)

In [20]:
ntt_paragraphs_path = '/Users/khanhnguyen/Document/QG/dataset/NTT_ViQuAD_not_standard/NTT_paragraphs.xlsx'
ntt_ps_xl = pd.ExcelFile(ntt_paragraphs_path)

data = []
for sheet_name in ntt_ps_xl.sheet_names:
    df = ntt_ps_xl.parse(sheet_name)
    for para in list(df['Context'].dropna()):
        d = {"paragraph": "", "type": "", "qa": []}
        d["type"] = sheet_name
        d["paragraph"] = para
        d["qa"] = ques_gen(para)
        data.append(d)

In [25]:
# Save data as json
ntt_viquad_not_standard = {"version": "ntt_viquad_not_standard", "data": data}
with open("/Users/khanhnguyen/Document/QG/dataset/NTT_ViQuAD_not_standard/NTT_ViQuAD_not_standard.json", "w+") as outfile:
    json.dump(ntt_viquad_not_standard, outfile)

In [40]:
data_test = data[:4]
df_nested_list = pd.json_normalize(data_test, meta=['paragraph', ['qa', 'sentence'], 'type'], record_path =['qa', 'qas'],)
df_nested_list.rename(columns = {'qa.sentence':'sentence'}, inplace = True)

df_nested_list.to_excel('excel.xlsx', 'sheet1')

In [53]:
df_data = pd.json_normalize(data, meta=['paragraph', ['qa', 'sentence'], 'type'], record_path =['qa', 'qas'])
df_data.rename(columns = {'qa.sentence':'sentence'}, inplace = True)

df_sheets = dict(tuple(df_data.groupby("type")))

# Write to excel
writer = pd.ExcelWriter('/Users/khanhnguyen/Document/QG/dataset/NTT_ViQuAD_not_standard/NTT_ViQuAD.xlsx', engine='xlsxwriter')
for sheet in df_sheets:
    df = df_sheets[sheet].iloc[:, [3,4,0,1,2]]
    df.to_excel(writer, sheet, index=False)

writer.save()

In [ ]:
# Get all paragraph
paragraphs = [{"paragraph": "", "type": ""}, {"paragraph": "", "type": ""}]
data = [{"paragraph": "", "type": "", "qa": []}, {"paragraph": "", "type": "", "qa": []}]